## AVISO:
Este notebook es provisional, la versión final estará en training.py

In [1]:
## Librerías:
# Para 'intents.json'
import json # Para el formato json
import codecs # Lectura de caracteres en español

# Procesamiento de Lenguaje Natural
import spacy # Lemmatizer (convertir palabras) con lenguaje español
import pickle
import numpy as np
import nltk # Natural Language ToolKit: Tokenizar

# Redes Neuronales:
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

In [2]:
# Preparación
nltk.download('punkt')

# Lematizador que acepta texto con Spacy español
nlp = spacy.load('es_core_news_sm')
def lemmatizer(text):  
    doc = nlp(text)
    return ' '.join([word.lemma_ for word in doc])

text = '¿personas?, ¿estás ahí?'
print(lemmatizer(text))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


¿ persona ? , ¿ estar ahí ?


In [3]:
# Cargar archivo json:
intents = json.loads(codecs.open('intents.json', encoding='utf-8').read())

In [4]:
intents['intents']

[{'tag': 'Saludos',
  'patterns': ['Hola',
   'Hey',
   'Hi',
   'Buenos días',
   'Buenas tardes',
   'Buenas noches',
   'Buenas',
   'Saludos'],
  'responses': ['Hola', 'Saludos', 'Hola, ¿cómo puedo ayudarte?']},
 {'tag': 'Consulta_Lugar',
  'patterns': ['¿Sabes donde está?',
   '¿Dónde queda?',
   '¿Dónde queda este lugar?',
   '¿Cómo puedo llegar?',
   '¿sabes dónde queda?',
   '¿me puedes llevar aqui?',
   'llévame aquí'],
  'responses': ['Ok, has hecho una consulta sobre <LUGAR>']},
 {'tag': 'Consulta_Tiempo',
  'patterns': ['¿Sabes cuándo empieza?',
   '¿Cuándo puedo ir?',
   '¿Desde qué hora está abierto?',
   '¿Qué días y horas?'],
  'responses': ['Ok, has hecho una consulta sobre <TIEMPO>']}]

In [19]:
def load_training_data(json_file = "intents.json"):
    # Cargar archivo json:
    intents = json.loads(codecs.open(json_file, encoding='utf-8').read())
    print("Archivo json cargado")
    
    words = [] # Palabras individuales usadas
    tags = [] # Etiquetas de intención
    ignore_letters = ['¿', '?', '.', '!', '(', ')']
    documents = [] # Lista de tuplas ([Lista de palabras], etiqueta asociada de intención)
    for intent in intents['intents']:
        # Por cada intención:
        for pattern in intent['patterns']:
            # Tokenizar el patrón de palabras que usaría el usuario y agregar las palabras a 'words'.
            pattern = lemmatizer(pattern)
            word_list = nltk.word_tokenize(pattern)
            words.extend(word_list)
            # Agregar a 'documents' la tupla (lista de palabras, etiqueta de la intención)
            documents.append((word_list, intent['tag']))
            if intent['tag'] not in tags:
                tags.append(intent['tag'])
            # Fin del segundo bucle
        # Fin del primer bucle
    words = [lemmatizer(word) for word in words if word not in ignore_letters]
    words = sorted(set(words))
    tags = sorted(set(tags))
    #print("Words: %s \n"%words)
    #print("Tags: %s \n"%tags)
    #print("Documents:", documents)
    return words, tags, documents

words, tags, documents = load_training_data()
print("W: %s"%words)
print("T: %s \n"%tags)
print("D:", documents)

Archivo json cargado
W: ['Buenas', 'Hey', 'Hi', 'Hola', 'abierto', 'aqui', 'aquí', 'buen', 'cuándo', 'cómo', 'desde', 'dimir', 'disponible', 'donde', 'día', 'dónde', 'empezar', 'estar', 'este', 'hora', 'ir', 'llegar', 'llevar', 'llévamar', 'lugar', 'noche', 'poder', 'quedar', 'querer', 'qué', 'saber', 'saludo', 'ser', 'tarde', 'y', 'yo']
T: ['Consulta_Lugar', 'Consulta_Tiempo', 'Saludos'] 

D: [(['Hola'], 'Saludos'), (['Hey'], 'Saludos'), (['Hi'], 'Saludos'), (['buen', 'día'], 'Saludos'), (['Buenas', 'tarde'], 'Saludos'), (['Buenas', 'noche'], 'Saludos'), (['Buenas'], 'Saludos'), (['saludo'], 'Saludos'), (['¿', 'saber', 'donde', 'estar', '?'], 'Consulta_Lugar'), (['¿', 'dónde', 'quedar', '?'], 'Consulta_Lugar'), (['¿', 'dónde', 'quedar', 'este', 'lugar', '?'], 'Consulta_Lugar'), (['¿', 'cómo', 'poder', 'llegar', '?'], 'Consulta_Lugar'), (['¿', 'saber', 'dónde', 'quedar', '?'], 'Consulta_Lugar'), (['¿', 'yo', 'poder', 'llevar', 'aqui', '?'], 'Consulta_Lugar'), (['llévamar', 'aquí'], 'Co

In [20]:
training = []
output_empty = [0] * len(tags)

for document in documents:
    bag = [] # bag of words
    word_patterns = document[0]
    word_patterns = [lemmatizer(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word.lower() in word_patterns else bag.append(0)
    
    output_row = list(output_empty)
    output_row[tags.index(document[1])] = 1
    training.append([bag, output_row])

#random.shuffle(training)
training = np.array(training)

train_x = list(training[:,0])
train_y = list(training[:,1])
print(len(train_x))
print(len(train_y))

for f in range(len(train_x)):
    if np.sum(train_x[f]) == 0:
        print("\n", f)
        print(train_x[f])
        print(train_y[f])

23
23

 6
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1]


In [21]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation = 'softmax'))

In [22]:
sgd = SGD(lr = 0.01, decay =1e-6, momentum=0.9, nesterov = True)

In [23]:
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


In [24]:
model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose = 1)

Train on 23 samples
Epoch 1/200
23/23 [==============================] - 5s 205ms/sample - loss: 1.1119 - accuracy: 0.3913
Epoch 2/200
23/23 [==============================] - 0s 1ms/sample - loss: 1.0866 - accuracy: 0.3913
Epoch 3/200
23/23 [==============================] - 0s 1ms/sample - loss: 1.0393 - accuracy: 0.4348
Epoch 4/200
23/23 [==============================] - 0s 2ms/sample - loss: 1.0291 - accuracy: 0.3478
Epoch 5/200
23/23 [==============================] - 0s 1ms/sample - loss: 0.9281 - accuracy: 0.5652
Epoch 6/200
23/23 [==============================] - 0s 1ms/sample - loss: 0.9028 - accuracy: 0.6957
Epoch 7/200
23/23 [==============================] - 0s 957us/sample - loss: 0.8609 - accuracy: 0.6087
Epoch 8/200
23/23 [==============================] - 0s 1ms/sample - loss: 0.8055 - accuracy: 0.6522
Epoch 9/200
23/23 [==============================] - 0s 1ms/sample - loss: 0.7747 - accuracy: 0.7826
Epoch 10/200
23/23 [==============================] - 0s 870us/samp